In [23]:
from mtg import *
from sentence_transformers import SentenceTransformer as ST
from sentence_transformers.util import pairwise_dot_score
from categorize import *
import pandas as pd

def dataset():
    data = pd.read_sql_query("""SELECT * 
    FROM cards 
    ORDER BY name;
    """,connectDB())
    data = data.drop_duplicates('name')
    data = data.dropna(axis=0, subset=['text'])
    return data

def cachedEmbeds(file=f"{homeDir}/data/text/text_vectors_dot_score.json"):
    return np.array(json.loads(open(file).read()))

def findSimilarCards(card,transformer,cardDataset,embeds,byCategory=False):
    if type(card) is Card:
        card = card.data
    text = transformer.encode(card['text'],normalize_embeddings=True)
    scores = pd.Series(pairwise_dot_score([text],embeds), index=cardDataset.index)
    cardDataset['scores'] = scores
    return cardDataset

def defaultModel():
    model = "bert-base-nli-mean-tokens"
    tf = ST(model,cache_folder=f"{homeDir}/data/text")
    return tf

In [ ]:
data = dataset()

In [ ]:
texts = data['text'].values

In [ ]:
model = defaultModel()

In [ ]:
embeds = model.encode(texts,normalize_embeddings=True)

In [2]:
import sqlalchemy as db
from mtg import *

In [43]:
engine = db.create_engine(f"sqlite:///{homeDir}/data/AllPrintings.sqlite")
conn = engine.connect()
meta = db.MetaData()

cards = db.Table("cards",meta,autoload=True,autoload_with=engine)
legalities = db.Table("legalities",meta,autoload=True,autoload_with=engine)



In [45]:
def commanderLegal():
    """
    SELECT * 
    FROM cards 
    LEFT JOIN legalities 
    ON legalities.uuid == cards.uuid 
    ORDER BY name
    """
    query = db.select([cards,legalities])
    query = query.select_from(
        cards.join(
            legalities, cards.c.uuid == legalities.c.uuid
        )
            ).where(
            db.and_(
                legalities.c.format == "commander", legalities.c.status == "Legal")
                ).order_by(cards.c.name)
    results = conn.execute(query).fetchall()
    #Create a DataFrame with the column names from the database.
    df = pd.DataFrame(results)
    df.columns = results[0].keys()
    #Drop duplicate cards by name, then drop cards with no rules text
    df = df.drop_duplicates('name')
    df = df.dropna(axis=0,subset=['text'])
    #The join inserts matching columns, in this case just UUID and ID, this probably isn't the optimal
    #way of querying for legality, but it works and this isn't terribly expensive to do after the fact.
    df.drop(['uuid_1','id_1'],axis=1,inplace=True)
    return df

In [46]:
comLegal = commanderLegal()